# Catégoriser automatiquement des questions

In [22]:
#pip install flask

In [23]:
#pip install -r requirements.txt

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
import joblib
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Définition du modèle Pydantic pour la saisie de texte
class InputText(BaseModel):
    text: str

# Définition de l'application FastAPI
app = FastAPI()

# Chargement du modèle BERT et du tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

# Fonction pour prédire les mots-clés
def predict_keywords(text: str) -> List[str]:
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits)
    predicted_labels = (probs > 0.5).cpu().detach().numpy().flatten()
    predicted_tags = mlb.inverse_transform([predicted_labels])[0]
    return predicted_tags

# Route pour la prédiction des mots-clés
@app.post("/predict/")
async def predict(input_text: InputText):
    try:
        keywords = predict_keywords(input_text.text)
        return {"keywords": keywords}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))